# Download data and organize in BIDS format

Instruction here: https://github.com/DCAN-Labs/abcd-dicom2bids

This repo has been cloned to `/Shared/tientong_scratch/abcd/code/00abcd2bids_convert/abcd2bids_github`

To run their scripts, make sure you follow what is listed in the file below
`/Shared/tientong_scratch/abcd/code/00abcd2bids_convert/abcd2bids_github/notes_abcd2bids.txt`

Also, because I only care about rs fMRI and sMRI, I went through all of the scripts and config files in the abcd-dicom2bids repo and delete everything related to task fMRI or validating BIDS (running BIDS validation needs docker, which can't be run on IFT machines). Below is an example of how to run this script

`python <path to script>/abcd2bids.py /Shared/pinc/sharedopt/apps/fsl/Linux/x86_64/5.0.8 /Shared/tientong_scratch/abcd/code/00abcd2bids_convert/R2016b/v91 --username tientong --password <your password> 2>&1 | tee abcd2bids_log.txt `

What I would recommend is submitting this as job on Argon

In [ ]:
# run this on argon

%%bash
qsub /Shared/tientong_scratch/abcd/code/00run_abcd2bids.sh 

## Move data

`mv /Shared/tientong_scratch/abcd/code/00abcd2bids_convert/abcd2bids_github/data/* /Shared/tientong_scratch/abcd/rawdata`

## Run BIDS Validator

http://bids-standard.github.io/bids-validator/

Some file wasn't associated with json files, e.g., `/Shared/tientong_scratch/abcd/rawdata/sub-NDARINV12F1NWCG/ses-baselineYear1Arm1/fmap/vol0002.nii.gz`

Right now not sure what happened with the abcd2bids script, move all of those files to 
`/Shared/tientong_scratch/abcd/nojson` with the code in the cell below:

In [ ]:
%%bash
for i in /Shared/tientong_scratch/abcd/rawdata/[a-zA-Z0-9]*/[a-zA-Z0-9]*/fmap ; do 
sub=(`echo $i | awk '{gsub("/"," "); print $5}' | awk '{gsub("-"," "); print $2}'`)
ses=(`echo $i | awk '{gsub("/"," "); print $6}' | awk '{gsub("-"," "); print $2}'`)
[ -f /Shared/tientong_scratch/abcd/rawdata/sub-${sub}/ses-${ses}/fmap/vol*2*.nii.gz ] && mkdir -p /Shared/tientong_scratch/abcd/nojson/sub-${sub}/ses-${ses}/fmap && mv /Shared/tientong_scratch/abcd/rawdata/sub-${sub}/ses-${ses}/fmap/vol*.nii.gz /Shared/tientong_scratch/abcd/nojson/sub-${sub}/ses-${ses}/fmap
done

## To run a different batch of participants

Download the `ABCD Fasttrack QC Instrument` file (https://nda.nih.gov/) with the list of participants you're interested in. You can then make changes to this file (e.g., exclude task fMRI, DTI) by running `/Shared/tientong_scratch/abcd/code/00abcd2bids_convert/fhpFastTrackQC/getStructandRS.R`

then, copy the output to

/Shared/tientong_scratch/abcd/code/00abcd2bids_convert/abcd2bids_github/spreadsheets/abcd_fastqc01.txt

## Multiple images of the same module of the same session

Have to pick the run that is better. How? running MRIQC


# Run MRIQC

## MRIQC Subject Level

Create a template file (currently put on argon

`/Users/tientong/job_scripts/mriqc/abcd/mriqc_TEMPLATE.job`

Content of the template file pasted below

In [ ]:
#!/bin/sh
#$ -pe smp 16
#$ -q PINC,CCOM,UI
#$ -m e
#$ -M tien-tong@uiowa.edu
#$ -o /Users/tientong/logs/abcd/mriqc/out
#$ -e /Users/tientong/logs/abcd/mriqc/err
OMP_NUM_THREADS=8
singularity run -H /Users/tientong/singularity_home \
/Users/tientong/mriqc_0.14.2.sif \
/Shared/tientong_scratch/abcd/rawdata \
/Shared/tientong_scratch/abcd/derivatives/mriqc \
participant --participant_label SUBJECT \
--no-sub --verbose-reports --write-graph \
-w /Shared/tientong_scratch/abcd/derivatives/mriqc \
--n_procs 8 --mem_gb 36000

In [ ]:
# create a list of available subjects

%%bash
outputFile=/Shared/tientong_scratch/abcd/log/subjectlist/200129list.txt
for i in /Shared/tientong_scratch/abcd/rawdata/[a-zA-Z0-9]*/[a-zA-Z0-9]*/anat ; do 
info=(`echo $i | awk '{gsub("/"," "); print $5}'`)
sub=(`echo $info | awk '{gsub("-"," "); print $2}'`)
echo $sub
done > ${outputFile}

In [ ]:
# then run this on argon to create individual mriqc job script

%%bash

for sub in $(cat /Shared/tientong_scratch/abcd/log/subjectlist/200129list.txt | tr '\n' ' ') ; do
sed -e "s|SUBJECT|${sub}|" /Users/tientong/job_scripts/mriqc/abcd/mriqc_TEMPLATE.job > /Users/tientong/job_scripts/mriqc/abcd/mriqc_sub-${sub}.job
done

for sub in $(cat /Shared/tientong_scratch/abcd/log/subjectlist/200129list.txt | tr '\n' ' ') ; do
    qsub /Users/tientong/job_scripts/mriqc/abcd/mriqc_sub-${sub}.job
done 

## After subject level mriqc

**Jan 30 2020** Completed running mriqc all subjects in the list below

`/Shared/tientong_scratch/abcd/log/subjectlist/200129list.txt`

EXCEPT  
sub-NDARINV93KR583V - T2w  
sub-NDARINVA01DRNBZ - resting state   
sub-NDARINVCVK8YHVR - T2w  
sub-NDARINVJXT9LVJP - T2w  
sub-NDARINVRT0YKWJM - T2w  
sub-NDARINVZL6CFV5G - T2w  
  
Detailed logs are at `/Shared/tientong_scratch/abcd/derivatives/mriqc/logs`

## MRIQC group level

In [ ]:
# after finish subject level analysis, run the group analysis (concatnate subjects' files)

singularity run -H /Users/tientong/singularity_home \
/Users/tientong/mriqc_0.14.2.sif \
/Shared/tientong_scratch/abcd/rawdata \
/Shared/tientong_scratch/abcd/derivatives/mriqc \
group --no-sub --verbose-reports --write-graph \
-w /Shared/tientong_scratch/abcd/derivatives/mriqc

Once the group level MRIQC is completed, move all files to

`/Shared/tientong_scratch/abcd/derivatives/mriqc/200129list_group`

# Choosing the best T1 or T2

Both BrainsTools and Freesurfer can only have 1 T1/T2 input, therefore, have to choose the run with the best quality before running sMRI analysis. 

Code: `/Shared/tientong_scratch/abcd/code/baw00ChooseBestStruct.R`

# Running BrainsTools

Follow the direction here https://github.com/TKoscik/nimg_core/blob/master/pipelines/structural_volumetrics/baw.md

1. Run the csv creator

`/Shared/tientong_scratch/abcd/abcd2bids_convert/abcd2bids_github/code/baw01BRAINSTools_csvcreator.sh`

Note: Looks like BrainsTools expects all session number to be different - have to replace session (baselineYear1Arm1) to a series of random numbers/strings 

In [ ]:
%%bash
for i in /Shared/tientong_scratch/abcd/rawdata/[a-zA-Z0-9]*/[a-zA-Z0-9]* ; do 
sub=(`echo $i | awk '{gsub("/"," "); print $5}'| awk '{gsub("-"," "); print $2}'`)
ses=(`echo $i | awk '{gsub("/"," "); print $6}'| awk '{gsub("-"," "); print $2}'`)
mv $i /Shared/tientong_scratch/abcd/rawdata/sub-${sub}/ses-$("uuidgen")
done

2. Make changes to the config file (on the lines written in the header

`/Shared/tientong_scratch/abcd/code/baw02BRAINSTools.config`

3. Run this BrainsTools on argon with

`/Shared/tientong_scratch/abcd/code/baw03runbaw.sh`

The steps are pasted below:

In [ ]:
# run this on argon
%%bash

sh /Shared/tientong_scratch/abcd/code/baw01BRAINSTools_csvcreator.sh -r /Shared/tientong_scratch -p abcd -o /Shared/tientong_scratch/abcd/derivatives/baw/200129.csv

export PATH=/Shared/pinc/sharedopt/apps/anaconda3/Linux/x86_64/4.3.0/bin:$PATH
bash /Shared/tientong_scratch/abcd/code/baw03runbaw.sh -p 1 -s all -r SGEGraph -c /Shared/tientong_scratch/abcd/code/baw02BRAINSTools.config

# Running Freesurfer

## First Create Atropos N4  T1 and T2

Template file is `/Shared/tientong_scratch/abcd/code/freesurfer00AtroposN4TEMPLATE.sh.in`, need to run a script to generate individuals' Atropos N4 correction script

In [ ]:
%%bash

cd /Shared/tientong_scratch/abcd/code/freesurfer00AtroposN4_T1 # all the sh scripts will be here
sh /Shared/tientong_scratch/abcd/code/freesurfer00createAtroposN4_T1.sh -r /Shared/tientong_scratch -p abcd
ls * | xargs -i{} -P10 sh {}

In [ ]:
%%bash

cd /Shared/tientong_scratch/abcd/code/freesurfer00AtroposN4_T2 # all the sh scripts will be here
sh /Shared/tientong_scratch/abcd/code/freesurfer00createAtroposN4_T2.sh -r /Shared/tientong_scratch -p abcd
ls * | xargs -i{} -P10 sh {}

In [ ]:
## Run Freesurfer for subjects with both T1 and T2

In [ ]:
%%bash

cd /Shared/tientong_scratch/abcd/code/freesurfer01createFSjobsT1T2 # all the sh scripts will be here
sh /Shared/tientong_scratch/abcd/code/freesurfer01createFSjobsT1T2.sh -r /Shared/tientong_scratch -p abcd
ls * | xargs -i{} -P10 sh {}

# Useful codes

In [ ]:
# dcm2niix code if needed

/Shared/pinc/sharedopt/apps/MRIcroGL/Linux/x86_64/1.2.20181114/dcm2niix -f %b_%d -z y -b y -o "/Shared/tientong_scratch/rawdata/sub-NDARINV01TXCDJN/ses-baselineYear1Arm1/anat/" /Shared/tientong_scratch/sourcedata/sub-NDARINV01TXCDJN/ses-baselineYear1Arm1/anat/*

#  -f : filename (%a=antenna (coil) name, 
#                 %b=basename, 
#                 %c=comments, 
#                 %d=description, 
#                 %e=echo number, 
#                 ### %f=folder name, 
#                 %i=ID of patient, 
#                 %j=seriesInstanceUID, 
#                 %k=studyInstanceUID, 
#                 %m=manufacturer, 
#                 %n=name of patient, 
#                 #### %p=protocol, 
#                 %r=instance number, 
#                 %s=series number, 
#                 #### %t=time, 
#                 %u=acquisition number, 
#                 %v=vendor, 
#                 %x=study ID; 
#                 %z=sequence name; 
#                 default '%s')
